In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import auc, roc_curve, roc_auc_score

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()


target = train['target']

train_input = train.drop(columns = ['ID_code', 'target'])
test_input = test.drop(columns = ['ID_code'])

X_train, X_test, Y_train, Y_test = train_test_split(train_input, target, test_size = 0.3, random_state=2019)


# Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression

logreg = LogisticRegression(class_weight = 'balanced')

logreg.fit(X_train, Y_train)

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight='balanced', dual=False,
          fit_intercept=True, intercept_scaling=1, max_iter=100,
          multi_class='warn', n_jobs=None, penalty='l2', random_state=None,
          solver='warn', tol=0.0001, verbose=0, warm_start=False)

In [53]:
logreg_pred = logreg.predict(X_test)

logreg_pred2 = logreg.predict_proba(X_test)[:,1]

Calculate accuracy for predicted classes.

In [54]:
fpr, tpr, thresholds = roc_curve(Y_test, logreg_pred, pos_label=1)
print('AUC:')
print(auc(fpr, tpr))

AUC:
0.7781260227624794


Calculate accuracy for predicted probabilities.

In [55]:
fpr, tpr, thresholds = roc_curve(Y_test, logreg_pred2, pos_label=1)
print('AUC:')
print(auc(fpr, tpr))

AUC:
0.8582668214033871


Submission format:

In [51]:
logreg_test_pred = logreg.predict_proba(test_input)[:,1]


submit = test[['ID_code']]
submit['target'] = logreg_test_pred

submit.head()

submit.to_csv('logreg.csv', index=False)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


# Decision Tree

In [56]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(class_weight='balanced')

dtc.fit(X_train, Y_train)

dtc_pred = dtc.predict_proba(X_test)[:,1]

fpr, tpr, thresholds = roc_curve(Y_test, dtc_pred, pos_label=1)
print('AUC:')
print(auc(fpr, tpr))

AUC:
0.5353729100184983


# LGBM

In [18]:
import lightgbm as lgb

'''params = {
           "metric" : "auc", 
           "objective" : "binary", 
           "class_weight" : "balanced", 
           "n_estimators" : 2000,
           "max_depth" : 15,
           "learning_rate" : 0.06,
           "num_leaves" : 100,
           "min_child_samples" : 100,
           "reg_alpha" : 1.728,
           "reg_lambda" : 4.98}
'''
params = {
    'bagging_freq': 5,
    'bagging_fraction': 0.335,
    'boost_from_average':'false',
    'boost': 'gbdt',
    'feature_fraction': 0.041,
    'learning_rate': 0.0083,
    'max_depth': -1,
    'metric':'auc',
    'min_data_in_leaf': 80,
    'min_sum_hessian_in_leaf': 10.0,
    'num_leaves': 13,
    'num_threads': 8,
    'tree_learner': 'serial',
    'objective': 'binary', 
    'verbosity': -1
}
lgtrain = lgb.Dataset(X_train, Y_train)
lgval = lgb.Dataset(X_test, Y_test)

model = lgb.train(params, lgtrain, valid_sets=[lgval])
model

[1]	valid_0's auc: 0.566354
[2]	valid_0's auc: 0.615919
[3]	valid_0's auc: 0.648804
[4]	valid_0's auc: 0.669663
[5]	valid_0's auc: 0.672095
[6]	valid_0's auc: 0.684419
[7]	valid_0's auc: 0.70154
[8]	valid_0's auc: 0.704246
[9]	valid_0's auc: 0.706915
[10]	valid_0's auc: 0.707846
[11]	valid_0's auc: 0.717052
[12]	valid_0's auc: 0.720023
[13]	valid_0's auc: 0.729281
[14]	valid_0's auc: 0.733461
[15]	valid_0's auc: 0.742749
[16]	valid_0's auc: 0.751069
[17]	valid_0's auc: 0.755887
[18]	valid_0's auc: 0.761148
[19]	valid_0's auc: 0.763096
[20]	valid_0's auc: 0.76089
[21]	valid_0's auc: 0.759461
[22]	valid_0's auc: 0.768696
[23]	valid_0's auc: 0.771542
[24]	valid_0's auc: 0.771696
[25]	valid_0's auc: 0.775956
[26]	valid_0's auc: 0.777587
[27]	valid_0's auc: 0.776559
[28]	valid_0's auc: 0.77467
[29]	valid_0's auc: 0.775856
[30]	valid_0's auc: 0.779549
[31]	valid_0's auc: 0.780272
[32]	valid_0's auc: 0.782844
[33]	valid_0's auc: 0.785563
[34]	valid_0's auc: 0.786245
[35]	valid_0's auc: 0.7914

In [ ]:
#model = lgb_model.fit(X_train, Y_train, eval_metric='auc',
#                  eval_set=[(X_test, Y_test)])

lgb_test_pred = model.predict(test_input, num_iteration=model.best_iteration)

#print('AUC: {}'.format(metrics.roc_auc_score(pred_classes, testing_classes)))

Submission format:

In [12]:
submitLGB = test[['ID_code']]
submitLGB["target"] = lgb_test_pred

submitLGB.to_csv('LGBM.csv', index = False)

submitLGB.head()

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ID_code,target
0,test_0,0.107390
1,test_1,0.079930
2,test_2,0.151206
3,test_3,0.140575
4,test_4,0.020527
